In [1]:
%run common.ipynb

In [61]:
import xml.etree.ElementTree as ET
import urllib

In [62]:
import sqlite3

con = sqlite3.connect('programs.db')
okpdtr_to_prog = dict(tqdm.tqdm(con.execute('select okpdtr.okpdtr, p.name from programs p left join program_okpdtr okpdtr on p.id=okpdtr.program')))
con.close()

8289it [00:00, 456314.29it/s]


In [65]:
def get_vacancies():
    evts = ET.iterparse('trudvsem/jobs.xml', events=('start', 'end'))
    root = None
    for evt, elem in evts:
        if elem.tag == 'vacancies':
            root = elem
        if evt == 'end' and elem.tag == 'vacancy':
            yield elem
            elem.clear()
            root.clear()
            
profs = get_vacancies()
profs = ((v.find('profession'), v.find('title'), v.findall('responsibilities'))
         for v in get_vacancies())
profs = (p for p in profs if p[0] is not None and p[1] is not None)
profs = ((urllib.parse.urlparse(url.attrib['resource']).fragment,
          name.text if name is not None else None,
          [r.text for r in resps])
         for (url, name, resps) in profs)
profs = ((okpdtr_to_prog[p[0]],) + p for p in profs if p[0] in okpdtr_to_prog)
profs = itertools.islice(profs, 10)
for p in profs:
    print(p)

('Биотехнология', '271421', 'Технолог', ['в соотвествии с должностной инструкцией для профессии расчетчик-технолог'])
('Экономика', '272443', 'Учитель', ['Планирует и осуществляет учебный процесс в соответствии с образовательной программой образовательного учреждения, разрабатывает рабочую программу по предмету'])
('Экономика', '272443', 'Учитель', ['Планирует и осуществляет учебный процесс в соответствии с образовательной программой образовательного учреждения, разрабатывает рабочую программу по предмету'])
('Экономика', '272443', 'Учитель', ['Планирует и осуществляет учебный процесс в соответствии с образовательной программой образовательного учреждения, разрабатывает рабочую программу по предмету'])
('Промышленное рыболовство', '224469', 'Инженер', ['Выполняет разработку проектов ЗСО (зон санитарной охраны) и СЗЗ (санитарно-защитных зон), составление экологической и гидрогеологической отчетности.'])
('Системный анализ и управление', '210659', 'Государственный инспектор по охране лес

In [58]:
! sqlite3 programs.db 'select programs.name from programs left join program_okpdtr on programs.id=program_okpdtr.program where program_okpdtr.okpdtr=184473;'